In [3]:
import polars as pl

In [123]:
df = pl.read_ipc("responses_70b_with_sizes.arrow", memory_map=False)

In [2]:
prompt = """The following is the final hundred words of a multiple choice response. The last character, excluding formatting like "**", must be a letter from A to J. In addition, the answer should be complete.

Respond directly without thinking, and use only the word "Invalid" or "Valid".

<example>

ATGGATAAAGTTTAAACAGAGAGGAATCT\n\nReversed: TCTAAGGAGAGCTGTTTAAAGTTTATGATGAT\n\nNow, take the complement of each base in this reversed sequence.\n\nT → A\n\nC → G\n\nT → A\n\nA → T\n\nA → T\n\nG → C\n\nG → C\n\nA → T\n\nG → C\n\nA → T\n\nG → C\n\nT → A\n\nG → C\n\nT → A\n\nT → A\n\nT → A\n\nA → T\n\nA → T\n\nG → C\n\nT → A\n\nT → A\n\nA → T\n\nT → A\n\nG → C\n\nA → T\n\nT → A\n\nSo the complement of the reversed antisense is:\n\nA G C T C G A C G C A T T T A T G C T T A T G"

</example>
Invalid

<example>
 privileged communications, option G is the clearest and most straightforward example of privilege without any complications or ambiguity. The woman’s consultation with the attorney about selling stock, including her financial disclosures, was clearly made for legal advice and remains privileged.\n\nAnswer: G
</example>
Valid

<example>
icient to indicate contract formed. But if it\'s a written confirmation, it is sufficient.\n\nD: Salesperson breached by not delivering. Not about statute of frauds.\n\nE: Memo is written contract. Not exactly, but perhaps close enough.\n\nF: Memo lacks signature. No, because under 2-201(2), the confirmation doesn\'t need to be signed by both.\n\nG: Statute of frauds not applicable between merchants. No, because it\'s still applicable with exceptions.\n\nH: Accepted before termination. Not relevant to statute.\n\nI: Salesperson didn\'t respond. Not directly relevant.\n\nSo the only option that aligns with the memo making the contract enforceable is E, but E says it\'s a written contract, which might not be accurate. Alternatively, perhaps the correct answer is E because the memo serves to take it out of the statute of frauds.\n\nWait, perhaps the memo is sufficient as a written contract because it indicates the existence of a contract, even without all terms. So E would be correct.\n\nWait, but the memo only refers to the discount and payment term, not quantity or price. So it doesn\'t include all the essential terms. Therefore, it\'s insufficient, making the salesperson\'s defense valid. So option C would be correct.\n\nSo I\'m torn between E and C.'
</example>
Invalid

<example>
{sample}
</example>
"""
 

In [9]:
import anthropic
from pathlib import Path
from anthropic.types.message_create_params import MessageCreateParamsNonStreaming
from anthropic.types.messages.batch_create_params import Request

client = anthropic.Anthropic(api_key=Path("anthropic.txt").read_text())

client.messages.batches.create(
    requests=[
        Request(
            custom_id="my-first-request",
            params=MessageCreateParamsNonStreaming(
                model="claude-3-7-sonnet-20250219",
                max_tokens=1024,
                messages=[{
                    "role": "user",
                    "content": "Hello, world",
                }]
            )
        ),
        Request(
            custom_id="my-second-request",
            params=MessageCreateParamsNonStreaming(
                model="claude-3-7-sonnet-20250219",
                max_tokens=1024,
                messages=[{
                    "role": "user",
                    "content": "Hi again, friend",
                }]
            )
        )
    ]
)

MessageBatch(id='msgbatch_01NV2xzv5HigukcDNhgGMQSb', archived_at=None, cancel_initiated_at=None, created_at=datetime.datetime(2025, 4, 7, 21, 8, 35, 405939, tzinfo=datetime.timezone.utc), ended_at=None, expires_at=datetime.datetime(2025, 4, 8, 21, 8, 35, 405939, tzinfo=datetime.timezone.utc), processing_status='in_progress', request_counts=MessageBatchRequestCounts(canceled=0, errored=0, expired=0, processing=2, succeeded=0), results_url=None, type='message_batch')

In [6]:
df["text"][0]

"Answer the following multiple choice question, selecting from the answer A through to J. After thinking, reply directly with your answer. \n\nThe last character of your response must be the letter you have chosen.\n\nQuestion:\n\nA property owner who owner her property in fee simple leased a three-story house and the 21 -acre tract upon which it stood to a friend for a term of 10 years. By the terms of the lease, the friend expressly covenanted to pay a monthly rental of $300 and to pay the taxes on the premises during the term of the lease. The lease also stipulated that the friend, as lessee, may not assign or sublease the said premises. The friend and his family lived in the house for two years. Then the friend assigned his leasehold interest in the property to a cousin, who was unaware of the prohibition against assignments. This written assignment expressly provided that the cousin would pay the owner the monthly rental of $300, but was silent concerning the payment of taxes. The

In [48]:
client.messages.count_tokens(
    model="claude-3-7-sonnet-20250219",
    messages=[
        {"role": "user", "content": prompt.format(sample=df["text"][0][-200:])}
    ]
)

MessageTokensCount(input_tokens=693)

In [20]:
df['text'][0][-100:]

'results in the apportionment of rent, as the lease is severed for the condemned portion. \n\nAnswer: D'

In [25]:
df["text"][-5]

"Answer the following multiple choice question, selecting from the answer A through to J. After thinking, reply directly with your answer. \n\nThe last character of your response must be the letter you have chosen.\n\nQuestion:\n\nWhich of the following statements about voting populations is most accurate?\n\nOptions:\n\nA: Voters are more likely to vote at age 18 than they are at age 65.\nB: An individual with a bachelor's degree is less likely to vote than an individual who has only a high school diploma.\nC: Affluent individuals are less likely to vote than impoverished individuals.\nD: Individuals are less likely to vote in a closely contested race.\nE: Individuals with no high school diploma are more likely to vote than those with a high school diploma.\nF: Individuals are less likely to vote if they live in a rural area than in an urban area.\nG: An individual with a graduate degree is more likely to vote than an individual who has only a high school diploma.\n\nAnswer:\n    <thi

In [27]:
client.messages.batches.create(
    requests=[
        Request(
            custom_id="my-first-request",
            params=MessageCreateParamsNonStreaming(
                model="claude-3-7-sonnet-20250219",
                max_tokens=1024,
                messages=[{
                    "role": "user",
                    "content": "Hello, world",
                }]
            )
        ),
        Request(
            custom_id="my-second-request",
            params=MessageCreateParamsNonStreaming(
                model="claude-3-7-sonnet-20250219",
                max_tokens=1024,
                messages=[{
                    "role": "user",
                    "content": "Hi again, friend",
                }]
            )
        )
    ]
)

MessageBatch(id='msgbatch_01P3zDxwWQCzy1aeUvtRTJf3', archived_at=None, cancel_initiated_at=None, created_at=datetime.datetime(2025, 4, 7, 21, 20, 50, 296784, tzinfo=datetime.timezone.utc), ended_at=None, expires_at=datetime.datetime(2025, 4, 8, 21, 20, 50, 296784, tzinfo=datetime.timezone.utc), processing_status='in_progress', request_counts=MessageBatchRequestCounts(canceled=0, errored=0, expired=0, processing=2, succeeded=0), results_url=None, type='message_batch')

In [50]:
requests = [
    Request(
        custom_id=str(i),
        params=MessageCreateParamsNonStreaming(
            model="claude-3-5-haiku-20241022",
            max_tokens=2,
            messages=[{
                "role": "user",
                "content": prompt.format(sample=df["text"][i][-200:]),
            }]
        )
    ) for i in range(len(df))
]

In [52]:
client.messages.batches.create(requests=requests)

MessageBatch(id='msgbatch_01JAgkpr8fcPn4xNab29MsUo', archived_at=None, cancel_initiated_at=None, created_at=datetime.datetime(2025, 4, 7, 21, 31, 57, 528318, tzinfo=datetime.timezone.utc), ended_at=None, expires_at=datetime.datetime(2025, 4, 8, 21, 31, 57, 528318, tzinfo=datetime.timezone.utc), processing_status='in_progress', request_counts=MessageBatchRequestCounts(canceled=0, errored=0, expired=0, processing=12214, succeeded=0), results_url=None, type='message_batch')

In [53]:
"msgbatch_01JAgkpr8fcPn4xNab29MsUo"

'msgbatch_01JAgkpr8fcPn4xNab29MsUo'

In [61]:
client.messages.batches.retrieve("msgbatch_01JAgkpr8fcPn4xNab29MsUo")

MessageBatch(id='msgbatch_01JAgkpr8fcPn4xNab29MsUo', archived_at=None, cancel_initiated_at=None, created_at=datetime.datetime(2025, 4, 7, 21, 31, 57, 528318, tzinfo=datetime.timezone.utc), ended_at=datetime.datetime(2025, 4, 7, 21, 39, 49, 127139, tzinfo=TzInfo(UTC)), expires_at=datetime.datetime(2025, 4, 8, 21, 31, 57, 528318, tzinfo=datetime.timezone.utc), processing_status='ended', request_counts=MessageBatchRequestCounts(canceled=0, errored=0, expired=0, processing=0, succeeded=12214), results_url='https://api.anthropic.com/v1/messages/batches/msgbatch_01JAgkpr8fcPn4xNab29MsUo/results', type='message_batch')

In [62]:
results = [result for result in client.messages.batches.results(
    "msgbatch_01JAgkpr8fcPn4xNab29MsUo",
)]

In [74]:
evaluations = {int(result.custom_id): result.result.message.content[0].text for result in results}

In [79]:
invalids = [k for k, v in evaluations.items() if v == "Invalid"]

In [87]:
df["text"][invalids[6]]

"Answer the following multiple choice question, selecting from the answer A through to J. After thinking, reply directly with your answer. \n\nThe last character of your response must be the letter you have chosen.\n\nQuestion:\n\nAn on-duty Police Officer hears breaking glass and a blaring alarm, and sees Stan run out of a darkened liquor store at 3 a.m. Stan runs around the corner, out of Officer's view, and hops into a car. Officer sees Stan drive away and pursues, lights flashing. Stan pulls over and Officer arrests him. Officer sees nothing in the passenger compartment, but pops the trunk and finds cash and a baseball bat. Stan's motion to suppress the cash and baseball bat as evidence will likely be\n\nOptions:\n\nA: Denied, because the Officer had a reasonable suspicion that Stan was involved in a crime.\nB: Granted, because the arrest was illegal and therefore the evidence is considered to be fruit of the poisonous tree.\nC: Granted, because the Officer did not have a search wa

In [99]:
[print("-"*30 + str(i) + "-"*30, df["text"][i][-200:]) for i in invalids];

------------------------------50------------------------------  derived from Gauss's law. Hence, this is not the answer.

The only quantity that decreases as the inverse square of distance is the electric field produced by a finite-length charged rod. 

Answer: D
------------------------------59------------------------------ of imperialism in his work "Imperialism: The Highest Stage of Capitalism," where he analyzed it as a late stage of capitalism characterized by global competition for colonies and resources.

Answer: F
------------------------------97------------------------------ n find V2.

Wait, but I don't have V2. Hmm.

Wait, perhaps I can compute the velocity at point 2 using the ideal gas law and the Bernoulli equation, but I don't have enough information.

Wait, maybe I
------------------------------121------------------------------ que is known as orgasmic reconditioning, which is used to help individuals alter their sexual responses by replacing less desirable fantasies wi

In [112]:
incorrectly_flagged = [50, 59, 121, 258, 293, 299, 697, 864, 1102, 1125, 1136, 1138, 1144, 1193, 1214, 1218, 1336, 1358, 1367, 1388, 1656, 1765, 1809, 1857, 1943, 2020, 2063,
                       2179, 2195, 2281, 2301, 2309, 2540, 2641, 2679, 2699, 2784, 2829, 2904, 2960, 2985, 3041, 3073, 3180, 3321, 3356, 3371,
                      3409, 3639, 3803, 4018, 4170, 4201, 4202, 4246, 4320, 4360, 4441, 4532, 4556, 4576, 4729, 4761, 4902, 5001, 5179, 5468, 5485, 5593, 5721, 5783, 5843,
                      5859, 5883, 6091, 6209, 6220, 6299, 6402, 6499, 6525, 6557, 6650, 6720, 6773, 6995, 7168, 7239, 7313, 7419, 7482, 7483, 7512, 
                      7694, 7858, 8022, 8027, 8057, 8141, 8184, 8186, 8279, 8460, 8499, 8591, 8739, 8789, 8840, 8968, 9450, 9476, 9539, 9672, 9894, 
                      9947, 10063, 10282, 10306, 10612, 10634, 10692, 
                      10769, 10829, 11062, 11077, 11151, 11176, 11329, 11347, 11503, 11520, 11739, 11780, 11812, 12031] # 1102?

In [113]:
[i for i, val in enumerate(incorrectly_flagged) if val not in invalids]

[]

In [117]:
reviewed_errors = [val for val in invalids if val not in incorrectly_flagged]

In [124]:
valid = [i not in reviewed_errors for i in range(len(df))]

In [126]:
df.insert_column(-1, pl.Series("valid", valid))

uuid,text,label,shape,first_think_token,valid,numeric label
str,str,str,list[i64],i64,bool,i64
"""responses-deepseek-ai-DeepSeek…","""Answer the following multiple …","""D""","[1, 2604]",387,true,3
"""responses-deepseek-ai-DeepSeek…","""Answer the following multiple …","""B""","[1, 3285]",149,true,1
"""responses-deepseek-ai-DeepSeek…","""Answer the following multiple …","""A""","[1, 767]",150,true,0
"""responses-deepseek-ai-DeepSeek…","""Answer the following multiple …","""C""","[1, 2529]",249,true,2
"""responses-deepseek-ai-DeepSeek…","""Answer the following multiple …","""H""","[1, 1950]",499,true,7
…,…,…,…,…,…,…
"""responses-deepseek-ai-DeepSeek…","""Answer the following multiple …","""G""","[1, 874]",198,true,6
"""responses-deepseek-ai-DeepSeek…","""Answer the following multiple …","""E""","[1, 795]",163,true,4
"""responses-deepseek-ai-DeepSeek…","""Answer the following multiple …","""B""","[1, 1259]",124,true,1


In [158]:
df.filter(pl.col("valid") ==  True).sort("shape", descending=True)["text"][20][-500:]

'{2} - \\frac{\\omega_e x_e}{4}\n\\]\n\n\\[\nG(0) = \\frac{0.1743}{2} - \\frac{0.0028768}{4}\n\\]\n\n\\[\nG(0) = 0.08715 \\, \\mathrm{eV} - 0.0007192 \\, \\mathrm{eV} = 0.0864308 \\, \\mathrm{eV}\n\\]\n\n### Step 3: Calculate \\( D_e \\)\nUsing \\( D_e = D_0 + G(0) \\):\n\n\\[\nD_e = 2.4287 \\, \\mathrm{eV} + 0.0864308 \\, \\mathrm{eV} = 2.51513 \\, \\mathrm{eV}\n\\]\n\n### Final Answer\nThe dissociation energy \\( D_e \\) for \\({ }^7 \\mathrm{Li}^1 \\mathrm{H}\\) is approximately \\( 2.5151 \\, \\mathrm{eV} \\), which corresponds to option **G**.\n\nG'

In [160]:
df.write_ipc("responses_70b_with_sizes.arrow")

In [ ]:
Selection deleted
